## Análise Exploratória do Conjunto de Dados

### Sumário <a class="anchor" id="topo"></a>

* [Parte 1: Configuração do notebook](#part_01)
* [Parte 2: Geração do arquivo .csv](#part_02)
    * [Parte 2.1: Listagem por Pessoa](#part_02_01)
    * [Parte 2.2: Listagem por Arquivo](#part_02_02)
* [Parte 3: Análise exploratória dos dados](#part_03)

### Parte 1: Configuração do notebook <a class="anchor" id="part_01"></a>

Nesta seção, as principais bibliotecas que serão usadas são importadas e as variáveis globais são definidas.

In [ ]:
import os

import pandas as pd

PATH_TO_DATASET = "/home/lozavival/Documents/AUDIOS-Dataset"

print("Done!")

### Parte 2: Geração do arquivo .csv <a class="anchor" id="part_02"></a>

Nesta seção, a estrutura de pastas do dataset é percorrida para criação de um arquivo .csv contendo informações de metadados dos arquivos presentes no dataset.

Esse arquivo pode ser gerado em duas versões: uma listagem por pessoa, em que cada linha da tabela contém informações sobre um locutor, ou uma listagem por arquivo, em que cada linha da tabela contém informações sobre um arquivo de áudio específico.

#### Versão 1: Listagem por Pessoa <a class="anchor" id="part_02_01"></a>

Aqui, o dataset é percorrido e é gerado um arquivo .csv com as seguintes informações de cada pessoa:

* Nome
* Código identificador
* Gênero (extraído do código identificador)
* Quantidade de arquivos de fala sintetizada
* Quantidade de arquivos de fala real

In [ ]:
# Create a dictionary containing the name, id, gender and number of spoofed
# and bonafide files for each person.
fake_audios_path = os.path.join(PATH_TO_DATASET, "fake_voices")
real_audios_path = os.path.join(PATH_TO_DATASET, "real_voices")
people = {}

# For every spoofed folder, get the number of spoofed files
for folder in os.listdir(fake_audios_path):
    path = os.path.join(fake_audios_path, folder)
    files = os.listdir(path)

    person, ids, *_ = folder.split("_")
    gender = ids[0]

    people[person] = {
        "gender": gender,
        "id": ids,
        "spoof_count": len(files),
    }

# For every bona-fide folder, get the number of bona-fide files
for folder in os.listdir(real_audios_path):
    path = os.path.join(real_audios_path, folder)
    files = os.listdir(path)

    person, ids, *_ = folder.split("_")
    gender = ids[0]

    if people.get(person) is not None:
        people[person]["bonafide_count"] = len(files)
    else:
        # If the person is not in the dictionary, there are no spoof files
        people[person] = {
            "gender": gender,
            "id": ids,
            "spoof_count": 0,
            "bonafide_count": len(files),
        }

print(len(people))
print(list(people.items())[:5])

In [ ]:
# Export the dictionary to a .csv file.
import csv

fields = ["person", "gender", "id", "spoof_count", "bonafide_count"]
with open(os.path.join(PATH_TO_DATASET, "meta.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    for person, data in people.items():
        writer.writerow([person, data["gender"], data["id"], data["spoof_count"], data["bonafide_count"]])
print("File written!")

#### Versão 2: Listagem por Arquivo <a class="anchor" id="part_02_02"></a>

O dataset é percorrido e é gerado um arquivo .csv com as seguintes informações de cada arquivo de áudio:

* Caminho relativo do arquivo no dataset
* Nome do locutor
* Código identificador do locutor
* Gênero do locutor (extraído de seu código identificador)
* Duração do áudio (em segundos)
* Rótulo do áudio ("spoof" ou "bona-fide")

In [ ]:
# Function to get the duration of an audio
import librosa

def get_duration(filename):
    try:
        audio_path = os.path.join(PATH_TO_DATASET, filename)
        y, sr = librosa.load(audio_path, sr=None)
        return librosa.get_duration(y=y, sr=sr)
    except Exception as e:
        print(f"Could not load file {filename}: {e}")
        return None

In [ ]:
# Create a list containing the path, speaker info, audio duration and label for each file.
fake_audios_path = os.path.join(PATH_TO_DATASET, "fake_voices")
real_audios_path = os.path.join(PATH_TO_DATASET, "real_voices")
all_files = []

# For every spoofed file, add its metadata to the list
for folder in os.listdir(fake_audios_path):
    folder_path = os.path.join(fake_audios_path, folder)
    folder_files = os.listdir(folder_path)

    person, ids, *_ = folder.split("_")
    gender = ids[0]
    for filename in folder_files:
        audio_path = os.path.join("fake_voices", folder, filename)  # relative path
        duration = get_duration(audio_path)
        if duration is not None:
            all_files.append([audio_path, person, ids, gender, duration, "spoof"])


# For every bona-fide file, add its metadata to the list
for folder in os.listdir(real_audios_path):
    path = os.path.join(real_audios_path, folder)
    folder_files = os.listdir(path)

    person, ids, *_ = folder.split("_")
    gender = ids[0]
    for filename in folder_files:
        audio_path = os.path.join("real_voices", folder, filename)
        duration = get_duration(audio_path)
        if duration is not None:
            all_files.append([audio_path, person, ids, gender, duration, "bona-fide"])


# Print the length of the list and the first 5 elements for visualization
print(len(all_files))
print(all_files[:5])

In [ ]:
# Export the list to a .csv file.
import csv

fields = ["file", "speaker", "id", "gender", "duration", "label"]
with open(os.path.join(PATH_TO_DATASET, "meta.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    writer.writerows(all_files)

print("File written!")

### Parte 3: Análise exploratória dos dados <a class="anchor" id="part_03"></a>

Nesta seção, o arquivo .csv contendo as informações de cada arquivo é carregado e é feita uma análise dos do conjunto de dados, em especial <completar com quais métricas serão avaliadas>.

In [ ]:
# Check if the .csv file exists
[x for x in os.listdir(PATH_TO_DATASET) if x.endswith('.csv')]

In [ ]:
# Load the .csv file in a pandas dataframe
dataset_metadata_df = pd.read_csv(os.path.join(PATH_TO_DATASET, 'meta.csv'), keep_default_na=False)
dataset_metadata_df.head()

In [ ]:
# Check if the dataframe is correct
assert len(dataset_metadata_df) == 179814
assert dataset_metadata_df.speaker.nunique() == dataset_metadata_df.id.nunique() == 101
assert dataset_metadata_df.gender.nunique() == 2
assert dataset_metadata_df.label.nunique() == 2
print("All tests passed!")

In [ ]:
print("total audio clips:", dataset_metadata_df.duration.count())
print("mean duration of audio clips (seconds):", dataset_metadata_df.duration.mean())
print("N speakers:", dataset_metadata_df.speaker.nunique())
print("Total audio time (hours):", dataset_metadata_df.duration.sum() / 3600)

In [ ]:
print("audio samples per speaker")
dataset_metadata_df.speaker.value_counts()

In [ ]:
print('Minutes per speaker')
dataset_metadata_df.groupby("speaker")['duration'].sum().sort_values(ascending=False) / 60

In [ ]:
print("samples by class")
dataset_metadata_df['label'].value_counts()

In [ ]:
print("minutes by class")
dataset_metadata_df.groupby("label")['duration'].sum().sort_values(ascending=False) / 60

In [ ]:
# the proportions of samples and duration are diferrent, why?
print("duration mean of each class")
print("mean duration of spoof audios is bigger")
dataset_metadata_df.groupby("label")['duration'].mean()

In [ ]:
print("Quantity of each audio per speaker")

spoof_per_speaker = dataset_metadata_df[dataset_metadata_df.label == 'spoof'].groupby("speaker").duration.count()
bonafide_per_speaker = dataset_metadata_df[dataset_metadata_df.label == 'bona-fide'].groupby("speaker").duration.count()

# Combine into a single DataFrame
counts_df = pd.DataFrame({
    'spoof_count': spoof_per_speaker,
    'bona_fide_count': bonafide_per_speaker
}).fillna(0)  # Fill NaN with 0 if some speakers have no 'spoof' or 'bona-fide' samples

# Calculate total count and ratio for each speaker
counts_df['total'] = counts_df['spoof_count'] + counts_df['bona_fide_count']
counts_df['spoof_ratio'] = counts_df['spoof_count'] / counts_df['total']

counts_df.sort_values("spoof_ratio")

In [ ]:
print("Minutes of each audio per speaker")

import pandas as pd

# Sum durations for each label per speaker, converting to minutes
spoof_per_speaker = dataset_metadata_df[dataset_metadata_df.label == 'spoof'].groupby("speaker").duration.sum() / 60
bonafide_per_speaker = dataset_metadata_df[dataset_metadata_df.label == 'bona-fide'].groupby("speaker").duration.sum() / 60

# Combine into a single DataFrame
duration_df = pd.DataFrame({
    'spoof_duration (min)': spoof_per_speaker,
    'bona_fide_duration (min)': bonafide_per_speaker
}).fillna(0)  # Fill NaN with 0 if some speakers have no 'spoof' or 'bona-fide' samples

# Calculate total duration and ratio for each speaker
duration_df['total_duration (min)'] = duration_df['spoof_duration (min)'] + duration_df['bona_fide_duration (min)']
duration_df['spoof_ratio'] = duration_df['spoof_duration (min)'] / duration_df['total_duration (min)']

# Sort by spoof ratio
duration_df = duration_df.sort_values("spoof_ratio")

duration_df.sort_values("spoof_ratio")
